In [ ]:
# pacotes

import requests
import pandas as pd
import json as json
import psycopg2
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os
import unicodedata


In [ ]:
# Caminho para o .env 
dotenv_path = "URL"

# Carrega as variáveis do arquivo .env, forçando substituição se já houver algo na memória
load_dotenv(dotenv_path, override=True)

# Recupera variáveis de ambiente
host = os.getenv("host")
porta = os.getenv("porta")
usuario = os.getenv("usuario")
senha = quote_plus(os.getenv("senha"))  # Protege caracteres especiais
banco = os.getenv("database")

# Mostra para confirmação
print("Conectando em:", host, porta, usuario, banco)

# Cria a engine
engine = create_engine(f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}")

In [ ]:
# Carrega o arquivo com as credenciais

with open('CREDENCIAIS') as f:
    config = json.load(f)

token = config['TOKEN_API']

In [ ]:
# API PEDIDOS

pagina_pedidos = 1
todos_dados_pedidos = []
limite_paginas = 1000000  

while True:
    url_pedidos = f"API/PEDIDOS"

    headers_pedidos = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_pedidos, headers=headers_pedidos)
    
    if response.status_code == 200:
        dados_pedidos = response.json()
        
        if not dados_pedidos:
            break

        todos_dados_pedidos.extend(dados_pedidos)
        print(f"✅ Página {pagina_pedidos} carregada. Total acumulado: {len(todos_dados_pedidos)}")

        pagina_pedidos += 1

        # Interrompe
        if pagina_pedidos > limite_paginas:
            print("🚧 Limite de páginas atingido (teste)")
            break

    else:
        print(f"❌ Erro na página {pagina_pedidos}: {response.status_code} - {response.text}")
        break

# Converte em DataFrame
df_pedidos = pd.DataFrame(todos_dados_pedidos)
print(df_pedidos.head())


In [ ]:
df_pedidos.head()

In [ ]:
# Função pra tirar os caracteres especiais das colunas de codigo e cor

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.strip().upper()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[^A-Z0-9]', '', texto)  # mantém apenas letras e números
    return texto

def limpar_coluna_texto(df, nome_coluna):
    df[nome_coluna] = df[nome_coluna].apply(normalizar_texto)
    return df

In [ ]:
df_pedidos = df_pedidos[['numero','ped_cli','dt_emissao','codcli','per_desc','vlr_desc','pgto','entrega','preco','valor','faturado','cancelado','valor_total_liq','valor_total_fat','valor_total_pen','valor_total_bruto']]

In [ ]:
df_pedidos.head()

In [ ]:
df_pedidos['dt_emissao'] = pd.to_datetime(df_pedidos['dt_emissao'], format="%d/%m/%Y", errors='coerce')

In [ ]:
#max data

data_max = df_pedidos['dt_emissao'].max()
print(data_max)

In [ ]:
#teste

teste_df = df_pedidos.query("dt_emissao == '2025-09-04' ")
teste_df.head()

In [ ]:
# rename

df_pedidos.rename(columns={
    'per_desc':'perc_desc',
    'numero':'codigo_pedido',
    'codcli':'codigo_cliente',
    'entrega':'dataPrevistaEntrega'
},inplace=True)

df_pedidos.columns

In [ ]:
# Converte a coluna de data corretamente para datetime
df_pedidos['dataPrevistaEntrega'] = pd.to_datetime(df_pedidos['dataPrevistaEntrega'], format="%d/%m/%Y", errors='coerce')

# Converte a coluna de data corretamente para datetime
df_pedidos['dt_emissao'] = pd.to_datetime(df_pedidos['dt_emissao'], format="%d/%m/%Y", errors='coerce')

In [ ]:
df_pedidos.head()

In [ ]:
df_pedidos.drop(columns=['dataPrevistaEntrega'],inplace=True)
df_pedidos.columns

In [ ]:
# Reflete a estrutura do banco para capturar a tabela já existen
metadata = MetaData()
metadata.reflect(bind=engine)
tabela_produtos = metadata.tables["FatoPedidos"]  # Tabela com nome e capitalização exatos

# Converte DataFrame em lista de dicionários
dados = df_pedidos.to_dict(orient="records")

# Cria comando de insert com fallback se houver conflito de PK
stmt = insert(tabela_produtos).values(dados)
stmt = stmt.on_conflict_do_nothing(index_elements=["codigo_pedido"])  # PK = codigo

# Executa a operação no banco
with engine.begin() as conn:
    conn.execute(stmt)
